In [3]:
import os
import win32com.client
from pywinauto import application
import time
import pandas as pd


def connect(reconnect=True):
    # 재연결이라면 기존 연결을 강제로 kill
    if reconnect:
        try:
            os.system('taskkill /IM ncStarter* /F /T')
            os.system('taskkill /IM CpStart* /F /T')
            os.system('taskkill /IM DibServer* /F /T')
            os.system('wmic process where "name like \'%ncStarter%\'" call terminate')
            os.system('wmic process where "name like \'%CpStart%\'" call terminate')
            os.system('wmic process where "name like \'%DibServer%\'" call terminate')
        except:
            pass

    CpCybos = win32com.client.Dispatch("CpUtil.CpCybos")

    if CpCybos.IsConnect:
        print('already connected.')

    else:
        app = application.Application()
        app.start(
            'C:\Daishin\Starter\\ncStarter.exe /prj:cp /id:{ssuyane} /pwd:{13579} /pwdcert:{pwdcert} /autostart'.format(
                id='ssuyane', pwd='13579', pwdcert='공인인증서패스워드')
        )
        # 연결 될때까지 무한루프
        while True:
            if CpCybos.IsConnect:
                break
            time.sleep(1)

        print('connected.')
    return CpCybos


def _get_ohlcv_by_date(StockChart, code, last_date, start_date, chart_gb, modified='1', field=(0, # 날짜
                                                                                               2, # 시가
                                                                                               3, # 고가
                                                                                               4, # 저가
                                                                                               5, # 종가
                                                                                               8, # 거래량
                                                                                               9, # 거래대금
                                                                                               12, # 상장주식수
                                                                                               13, # 시가총액
                                                                                               16, # 외국인보유수량
                                                                                               17, # 외국인보유비율
                                                                                               20, # 기관순매수
                                                                                               21, # 기관누적순매수
                                                                                               23 # 등락률
                                                                                               )):
    StockChart.SetInputValue(0, code)
    StockChart.SetInputValue(1, ord('1'))
    StockChart.SetInputValue(2, last_date)
    StockChart.SetInputValue(3, start_date)
    StockChart.SetInputValue(5, field)
    StockChart.SetInputValue(6, ord(chart_gb))
    StockChart.SetInputValue(9, ord(modified))
    StockChart.BlockRequest()
    return _get_df_from_stock_chart(StockChart)

def initIndexInfo(self):
    # 지표 기본 정보, 지표 이름 = 지표 라인 리스트
    self.indexList = {}
    self.indexList['지표선택 없음'] =['없음']
    self.indexList['이동평균(라인1개)'] =['이동평균']
    self.indexList['Stochastic Slow'] = ['SLOW K', 'SLOW D']
    self.indexList['MACD'] = ['MACD', 'SIGNAL', 'OSCILLATOR']
    self.indexList['RSI'] = ['RSI', 'SIGNAL']
    self.indexList['Binary Wave MACD'] = ['BWMACD','SIGNAL', 'OSCILLATOR']
    self.indexList['TSF'] = ['TSF','SIGNAL']
    self.indexList['ZigZag'] = ['ZigZag1', 'ZigZag2']
    self.indexList['Bollinger Band'] = ['Bol상', 'Bol하', 'Bol중']

def _get_df_from_stock_chart(StockChart):
    time.sleep(0.25)
    df = []
    num_col = StockChart.GetHeaderValue(1)
    num_row = StockChart.GetHeaderValue(3)
    code = StockChart.GetHeaderValue(0) # 초
    timess = StockChart.GetHeaderValue(18) # 초
    exFlag = StockChart.GetHeaderValue(19) # 예상체결 플래그
    cprice = StockChart.GetHeaderValue(13) # 현재가
    diff = StockChart.GetHeaderValue(2) # 대비
    cVol = StockChart.GetHeaderValue(17) # 순간체결수량
    vol = StockChart.GetHeaderValue(9) # 거래량
    
    for r_idx in range(num_row):
        curr_data = []

        for c_idx in range(num_col):
            curr_data.append(StockChart.GetDataValue(c_idx, r_idx))
        df.append(curr_data)

    df = pd.DataFrame(df, columns=['date', 'time', 'open', 'high', 'low', 'close', 'volume', 'Accumulated sales volume' , 'Accumulated number of contract'])
    return df


def get_ohlcv_by_date(code, last_date, start_date, chart_gb, modified='1', field=(0,1,2,3,4,5,8,10,11)):
    '''
    :param code: 종목코드
    :param last_date: 마지막 날짜
    :param start_date: 시작 날짜
    :param chart_gb: 
    'D' = 일
    'W' = 주
    'M' = 월
    'm' = 분
    'T' = 틱
    :param modified:
    '0' = 무수정주가
    '1' = 수정주가
    :param field:
    :return:
    '''

    print('get ohlcv by date')
    print(f'code: {code}\nstart_date: {start_date}\nlast_date: {last_date}')
    StockChart = win32com.client.Dispatch("CpSysDib.StockChart")

    df = _get_ohlcv_by_date(StockChart, code, last_date, start_date,
                            chart_gb, modified=modified, field=field)

    last_date = df['date'].iloc[-1]
    last_min = df['time'].iloc[-1]

    df_list = [df]

    while True:
        df = _get_ohlcv_by_date(StockChart, code, int(last_date), start_date,
                            chart_gb, modified=modified, field=field)

        curr_date = df['date'].iloc[-1]
        curr_min = df['time'].iloc[-1]

        if curr_date == last_date and curr_min == last_min:
            break

        last_date = curr_date
        last_min = curr_min

        df_list.append(df)

    df = pd.concat(df_list).drop_duplicates(['date', 'time']).reset_index(drop=True)
    return df


# 이미 연결되어있다면 재연결 x
CpCybos = connect(False)

# kospi200_code =['005930', '373220', '000660', '207940', '006400', '051910', '005380', '000270', '005490', '035420', '003670', '035720', '012330', '068270', '066570', '105560', '028260', '055550', '096770', '032830', '003550', '015760', '329180', '352820', '033780', '086790', '034020', '323410', '000810', '009150', '034730', '017670', '003490', '011200', '018260', '042660', '010130', '138040', '009540', '259960', '316140', '024110', '047050', '030200', '086280', '010950', '011070', '009830', '051900', '012450', '361610', '011170', '402340', '010140', '377300', '036570', '241560', '326030', '302440', '028050', '090430', '034220', '047810', '005830', '271560', '001570', '018880', '267250', '161390', '032640', '004020', '000100', '383220', '006800', '064350', '097950', '011790', '251270', '000720', '011780', '035250', '128940', '112610', '010620', '029780', '078930', '021240', '016360', '005940', '180640', '008770', '282330', '006260', '272210', '111770', '052690', '071050', '001450', '000990', '004990', '028670', '020150', '004370', '204320', '003410', '010120', '014680', '042670', '007070', '081660', '036460', '039490', '138930', '000880', '008930', '139480', '088350', '002790', '030000', '012750', '005420', '298050', '005850', '079550', '026960', '023530', '011210', '004170', '336260', '001040', '010060', '004000', '002380', '001440', '175330', '000120', '047040', '017800', '007310', '298020', '000080', '051600', '000150', '120110', '001120', '073240', '006280', '139130', '009900', '137310', '004800', '375500', '001740', '069960', '093370', '006360', '005300', '000240', '285130', '103140', '271940', '069620', '161890', '014820', '009420', '178920', '192820', '009240', '000670', '185750', '010780', '280360', '003230', '114090', '001800', '020560', '016380', '006650', '069260', '300720', '019170', '105630', '039130', '000210', '192080', '008730', '032350', '003090', '294870', '004490', '381970', '003240', '005250', '031430', '001680', '020000', '003850', '057050', '284740', '013890']
kospi200_code = ['A012450'] #, 'A373220', 'A000660', 'A207940', 'A006400', 'A051910', 'A005380', 'A000270', 'A005490', 'A035420', 'A003670', 'A035720', 'A012330', 'A068270', 'A066570', 'A105560', 'A028260', 'A055550', 'A096770', 'A032830', 'A003550', 'A015760', 'A329180', 'A352820', 'A033780', 'A086790', 'A034020', 'A323410', 'A000810', 'A009150', 'A034730', 'A017670', 'A003490', 'A011200', 'A018260', 'A042660', 'A010130', 'A138040', 'A009540', 'A259960', 'A316140', 'A024110', 'A047050', 'A030200', 'A086280', 'A010950', 'A011070', 'A009830', 'A051900', 'A012450', 'A361610', 'A011170', 'A402340', 'A010140', 'A377300', 'A036570', 'A241560', 'A326030', 'A302440', 'A028050', 'A090430', 'A034220', 'A047810', 'A005830', 'A271560', 'A001570', 'A018880', 'A267250', 'A161390', 'A032640', 'A004020', 'A000100', 'A383220', 'A006800', 'A064350', 'A097950', 'A011790', 'A251270', 'A000720', 'A011780', 'A035250', 'A128940', 'A112610', 'A010620', 'A029780', 'A078930', 'A021240', 'A016360', 'A005940', 'A180640', 'A008770', 'A282330', 'A006260', 'A272210', 'A111770', 'A052690', 'A071050', 'A001450', 'A000990', 'A004990', 'A028670', 'A020150', 'A004370', 'A204320', 'A003410', 'A010120', 'A014680', 'A042670', 'A007070', 'A081660', 'A036460', 'A039490', 'A138930', 'A000880', 'A008930', 'A139480', 'A088350', 'A002790', 'A030000', 'A012750', 'A005420', 'A298050', 'A005850', 'A079550', 'A026960', 'A023530', 'A011210', 'A004170', 'A336260', 'A001040', 'A010060', 'A004000', 'A002380', 'A001440', 'A175330', 'A000120', 'A047040', 'A017800', 'A007310', 'A298020', 'A000080', 'A051600', 'A000150', 'A120110', 'A001120', 'A073240', 'A006280', 'A139130', 'A009900', 'A137310', 'A004800', 'A375500', 'A001740', 'A069960', 'A093370', 'A006360', 'A005300', 'A000240', 'A285130', 'A103140', 'A271940', 'A069620', 'A161890', 'A014820', 'A009420', 'A178920', 'A192820', 'A009240', 'A000670', 'A185750', 'A010780', 'A280360', 'A003230', 'A114090', 'A001800', 'A020560', 'A016380', 'A006650', 'A069260', 'A300720', 'A019170', 'A105630', 'A039130', 'A000210', 'A192080', 'A008730', 'A032350', 'A003090', 'A294870', 'A004490', 'A381970', 'A003240', 'A005250', 'A031430', 'A001680', 'A020000', 'A003850', 'A057050', 'A284740', 'A013890']
for i in kospi200_code:
    code = i
    # 분봉은 최대 2년 데이터 제공
    start_date = '20210627'
    last_date = '20230627'
    chart_gb = 'm'
    df = get_ohlcv_by_date(code, last_date, start_date, chart_gb)
    print(df)
    df.to_csv(f'C:\\big16\\final_project\\DATA\\KOSPI200_JONGMOK\{i}분봉데이터.csv')

already connected.
get ohlcv by date
code: A012450
start_date: 20210627
last_date: 20230627
            date  time    open    high     low   close  volume  \
0       20230627  1530  130500  130500  130500  130500   14902   
1       20230627  1520  130300  130300  130200  130300    2576   
2       20230627  1519  130200  130300  130200  130200     847   
3       20230627  1518  130200  130300  130100  130300    2203   
4       20230627  1517  130300  130300  130100  130200     876   
...          ...   ...     ...     ...     ...     ...     ...   
187304  20210630   905   47200   47250   47150   47200    1224   
187305  20210630   904   47200   47200   47150   47150     941   
187306  20210630   903   47200   47200   47150   47200    1170   
187307  20210630   902   47350   47400   47200   47200    1382   
187308  20210630   901   47450   47500   47350   47400    3572   

        Accumulated sales volume  Accumulated number of contract  
0                         241033                